In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    zoom_range = 0.2,
    brightness_range = (0.5,1.5),
    channel_shift_range=0.5,
    rotation_range = 20,
    horizontal_flip=True,
)
train_dir = '../datasets/train'

train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size = (150,150),
    class_mode = 'categorical',
    batch_size = 50
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)

test_dir = '../datasets/test'

test_dataset = test_datagen.flow_from_directory(
    test_dir,
    target_size = (150,150),
    class_mode = 'categorical',
    batch_size = 50
)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (150,150,3), padding = 'same'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(128, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(3, activation = 'softmax')
])

model.summary()

In [ ]:
class MyCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, log={}):
        if (log.get('accuracy') > .95 and log.get('val_accuracy') > .93):
            print("\n 93% accuray reached cancelling training")
            self.model.stop_training = True

callbacks = MyCallback()

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_dataset,
    validation_data = test_dataset,
    steps_per_epoch = 30,
    validation_steps = 20,
    epochs = 30,
    callbacks = [callbacks]
)

In [ ]:
import os
path='../datasets/test/'
choices = ['paper', 'scissor', 'stone']
test_size = 70

for choice in choices:
    
    nPath = path + choice + '/'
    count = 0
    
    for fil in os.listdir(nPath):
        
        file = nPath + fil
        
        img=image.load_img(file, target_size=(150, 150))

        x=image.img_to_array(img)
        x=np.expand_dims(x, axis=0)
        x = x/255
        images = np.vstack([x])

        classes = model.predict(images)
        pre = choices[np.where(classes == np.amax(classes))[1][0]]
        
        if (letter == pre):
            count += 1
    print("Class {} : {}".format(choice, count/test_size))

In [ ]:
model_json = model.to_json()
with open('modelFinal.json','w') as json_file:
    json_file.write(model_json)

model.save_weights('modelWeights.h5')